# Clean data and update airports collection

In [1]:
#mongoimport --db companies -c points --collection airports --file INPUT/airports.geojson
from pymongo import MongoClient
import pandas as pd
import functions as F

In [2]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()
airports = db.airports

In [3]:
df = pd.DataFrame(list(airports.find({})))
df = F.CleanDataFrame(df,'features')
dfexp = df[['properties']].apply(lambda r: r['properties'], result_type="expand", axis=1)
df = pd.concat([df,dfexp], axis=1)

In [4]:
df = df.rename(columns={"location":"terminal","geometry":"location"}).reset_index()
df = df.drop(columns =['_id','features','bbox','type','properties','wikipedia','featurecla','index','terminal'])

,location,scalerank,name,abbrev,gps_code,iata_code,natlscale
0,"{'type': 'Point', 'coordinates': [75.957072, 3...",9,Sahnewal,LUH,VILD,LUH,8
1,"{'type': 'Point', 'coordinates': [75.93306, 17...",9,Solapur,SSE,VASL,SSE,8
2,"{'type': 'Point', 'coordinates': [85.323597, 2...",9,Birsa Munda,IXR,VERC,IXR,8
3,"{'type': 'Point', 'coordinates': [48.747107, 3...",9,Ahwaz,AWZ,OIAW,AWZ,8
4,"{'type': 'Point', 'coordinates': [78.217219, 2...",9,Gwalior,GWL,VIGR,GWL,8
...,...,...,...,...,...,...,...
886,"{'type': 'Point', 'coordinates': [-3.569027, 4...",2,Madrid Barajas,MAD,LEMD,MAD,150
887,"{'type': 'Point', 'coordinates': [-66.00423, 1...",2,Luis Munoz Marin,SJU,TJSJ,SJU,150
888,"{'type': 'Point', 'coordinates': [17.93073, 59...",2,Arlanda,ARN,ESSA,ARN,150
889,"{'type': 'Point', 'coordinates': [106.654296, ...",2,Soekarno-Hatta Int'l,CGK,WIII,CGK,150


In [5]:
df.to_json("../OUTPUT/airports.json", orient="records")
#mongoimport --db companies --collection airports --file OUTPUT/airports.json --jsonArray